In [1]:
"""importing libararies"""
import pandas as pd
import numpy as np
import seaborn
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import StandardScaler 
import warnings
import tensorflow as tf
import pickle
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import model_from_json
import ifxdaq
import processing
from ifxdaq.sensor.radar_ifx import RadarIfxAvian
from scipy import signal
from scipy.signal import find_peaks
warnings.filterwarnings('ignore')

In [2]:
def loading_model_parameters():
  # loading the tranied model standard scaler parameters
   scaler = pickle.load(open('scaler.pkl','rb'))    

  # loading the tranied model PCA parameters
   pca = pickle.load(open('pca_req.pkl','rb'))      

  # loading the ann model parameters
   file = open('model.json', 'r')
   loaded  = file.read()
   file.close()
   ann_model = model_from_json(loaded)
   ann_model.load_weights("model.h5")

   # loading the radar configurations
   config_file = "radar_configs/RadarIfxBGT60.json"

   return scaler, pca, ann_model


In [ ]:
def fetch_data(time_ms=10000):
    raw_data    = []
    with RadarIfxAvian(config_file) as device:                             # Initialize the radar with configurations
        for i_frame, frame in enumerate(device):                           # Loop through the frames coming from the radar
            raw_data.append(np.squeeze(frame['radar'].data/(4095.0)))      # Dividing by 4095.0 to scale the data
            
            if(len(raw_data) > time_ms-1 and len(raw_data) % time_ms == 0):        # 5000 is the number of frames. which corresponds to 5seconds
                data = np.swapaxes(np.asarray(raw_data), 0, 1)
                
                phases, abses, centers, radii = processing.do_processing(data)       # preprocessing to get the phase information
                range_idx, range_data = processing.processing_rangeData(data)        # to the get range_fft data

                phases = np.mean(phases, axis=0)
                break
                
    return phases, range_data

In [ ]:
def detect_state(data):
  '''checking the state of the person : Anxious and Rest'''
  sc,pc,ann_model = loading_model_parameters()
  scaled_data = sc.transform(data.T)
  test_pca = pc.transform(scaled_data)
  y_pred=ann_model.predict(test_pca)

  if(y_pred>=0.5):
    print("Anxious")
  else:
    print("Rest")

  return None

In [3]:
def get_vitals(phases_data):
  displacement = (0.005/(4*np.pi))*phases_data

  # band pass filter for breathrate
  sos_b = signal.butter(6, (0.2/500,0.5/500), 'band', fs=1, output='sos')
  filtered_b = signal.sosfilt(sos_b, displacement)

  peaks_b,_= find_peaks(filtered_b, prominence=(0.0005, None))
  numpeaks_b=len(peaks_b)
  breathrate=numpeaks_b*6

  # band pass filter for heartrate

  sos_h = signal.butter(6, (0.5/500,2/500), 'band', fs=1, output='sos')
  filtered_h = signal.sosfilt(sos_h, displacement)
  peaks_h,_= find_peaks(filtered_h, prominence=(0.00020,None))
  numpeaks_h=len(peaks_h)
  heartrate=numpeaks_h*6
  
  return breathrate, heartrate